### Задание:

Написать функцию, которая принимает слово и находит ближайшее к нему в словаре (сгенерируйте несколько кандидатов с помощью get_closest_match_vec, а затем посчитать метрики близости до каждого слова и вывести самое близкое). 

совместить get_closest_match_vec (находит варианты исправлений по символьному векторному представлению слов. Однако она очень плохо учитывает порядок символов в слове) и левенштейна (находит более точные исправления, но работает долго).

Новая функция должна находить несколько (3-10) вариантов по векторной близости, а затем для каждого из вариантов рассчитывать расстояние левенштейна и выдавать лучший. 

In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from difflib import get_close_matches
import textdistance
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
punct = set(punctuation)
from sklearn.metrics import classification_report
from string import punctuation

### Решение:

In [2]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)
    
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,4)) #улучшаю качество через ngram_range=(1,4)
X = vec.fit_transform(vocab)

def get_closest_hybrid_match(text, X, vec, metric=textdistance.levenshtein):
    TOPN=3
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    a = [id2word[top] for top in topn]
        
    similarities = Counter()
    for word in a:
        similarities[word] = metric.normalized_similarity(text, word) 
            
    
    return similarities.most_common(1)[0]

In [3]:
%%time
get_closest_hybrid_match('грсцдарство', X, vec)

Wall time: 230 ms


('государство', 0.8181818181818181)

Проверить качество работы такой функции на данных соревнования Диалога. 

In [33]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

correct = 0
total = 0
mistakes = []

for i in range(100):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        predict = get_closest_hybrid_match(pair[1], X, vec)[0]
        if predict == pair[0]:
            correct += 1
        
        else:
            a = [predict, pair[0]]
            mistakes.append(a)
            
            
        total += 1

In [34]:
print ('quality =', str(correct/total)[2:4],'%')

quality = 80 %


Посмотреть на ошибки

In [35]:
pprint(mistakes) #1 - то, как исправила моя функция, 2 - правильное слово

[['симпатичный', 'симпатичнейшее'],
 ['шпионской', 'шпионское'],
 ['гламур', 'гламурный'],
 ['бонди', 'бонда'],
 ['superjet', 'superheadz'],
 ['ap', 'clap'],
 ['tamer', 'camera'],
 ['стояния', 'поясним'],
 ['ополчатся', 'получатся'],
 ['язычка', 'язычки'],
 ['милы', 'милые'],
 ['нащокина', 'насчет'],
 ['чавес', 'чавеса'],
 ['основа', 'основная'],
 ['попавшие', 'попавшим'],
 ['поисково-спасательных', 'аварийно-спасательных'],
 ['краснов', 'напрасно'],
 ['общем', 'в'],
 ['как', 'общем'],
 ['вы', 'как'],
 ['знаете', 'вы'],
 ['из', 'знаете'],
 ['моего', 'из'],
 ['не', 'моего'],
 ['давнего', 'недавнего'],
 ['пропал', 'пропажу'],
 ['щекам', 'ящика'],
 ['почте', 'почте.ру'],
 ['хорошей', 'хорошо'],
 ['выход', 'выходных'],
 ['крите', 'рите'],
 ['датам', 'потому'],
 ['штат', 'что'],
 ['упреждать', 'переждать'],
 ['дубрава', 'дубраве'],
 ['алюминия', 'люминала'],
 ['тыка', 'повтыкав'],
 ['летным', 'билетным'],
 ['касса', 'кассам'],
 ['тото', 'что-то'],
 ['тамошний', 'мощный'],
 ['рабочем', 'нера

In [39]:
print('Процент ошибок:', len(mistakes)/total)

Процент ошибок: 0.1926192619261926
